In [7]:
%load_ext autoreload
%autoreload 2

import misc
import config

layers_indexes = config.get_layers_indexes(4)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
genome = [1,0,0,0,1,0]
module_genome = misc.module_convert(genome, layers_indexes)
module_graph = misc.get_graph(module_genome)


{1: [], 2: [1], 3: [], 4: [2]}


In [9]:
print(module_genome)
print(module_graph)

[[1], [0, 0], [0, 1, 0]]
{1: [], 2: [1], 3: [], 4: [2], 5: [2, 4]}
